In [29]:
# Automatically reload imported modules that are changed outside this notebook
%load_ext autoreload
%autoreload 2

# More pixels in figures
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.dpi"] = 200

# Init PRNG with fixed seed for reproducibility
import numpy as np
np_rng = np.random.default_rng(1)

import tensorflow as tf
tf.random.set_seed(np_rng.integers(0, tf.int64.max))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
!python3 --version

Python 3.7.11


# Common Voice spoken language identification with a  neural network

**2020-11-08**


This example is a thorough, but simple walk-through on how to do everything from loading mp3-files containing speech to preprocessing and transforming the speech data into something we can feed to a neural network classifier.
Deep learning based speech analysis is a vast research topic and there are countless techniques that could possibly be applied to improve the results of this example.
This example tries to avoid going into too much detail into these techniques and instead focuses on getting an end-to-end classification pipeline up and running with a small dataset.

## Data

This example uses open speech data downloaded from the [Mozilla Common Voice](https://commonvoice.mozilla.org/en/datasets) project.
See the readme file for downloading the data.
In addition to the space needed for the downloaded data, you will need at least 10 GiB of free disk space for caching (can be disabled).

In [39]:
import urllib.parse
from IPython.display import display, Markdown


languages = """
fi
""".split()

languages = sorted(l.strip() for l in languages)

display(Markdown("### Languages"))
display(Markdown('\n'.join("* `{}`".format(l) for l in languages)))

bcp47_validator_url = 'https://schneegans.de/lv/?tags='
display(Markdown("See [this tool]({}) for a description of the BCP-47 language codes."
                 .format(bcp47_validator_url + urllib.parse.quote('\n'.join(languages)))))

### Languages

* `fi`

See [this tool](https://schneegans.de/lv/?tags=fi) for a description of the BCP-47 language codes.

## Loading the metadata

We start by preprocessing the Common Voice metadata files.

Update `datadir` and `workdir` to match your setup.
All output will be written to `workdir`.

In [40]:
import os


workdir = "data/"
#datadir = "/mnt/data/speech/common-voice/downloads/2020/cv-corpus"
datadir = "/Users/haily/Downloads/cv-corpus-7.0-2021-07-21"
""
print("work dir:", workdir)
print("data source dir:", datadir)

os.makedirs(workdir, exist_ok=True)
assert os.path.isdir(datadir), datadir + " does not exist"

work dir: data/
data source dir: /Users/haily/Downloads/cv-corpus-7.0-2021-07-21


Common Voice metadata is distributed as `tsv` files and all audio samples are mp3-files under `clips`.

In [41]:
dirs = sorted((f for f in os.scandir(datadir) if f.is_dir()), key=lambda f: f.name)

print(datadir)
for d in dirs:
    if d.name in languages:
        print(' ', d.name)
        for f in os.scandir(d):
            print('   ', f.name)

missing_languages = set(languages) - set(d.name for d in dirs)
assert missing_languages == set(), "missing languages: {}".format(missing_languages)

/Users/haily/Downloads/cv-corpus-7.0-2021-07-21
  fi
    .DS_Store
    clips
    train.tsv
    dev.tsv
    validated.tsv
    test.tsv
    other.tsv
    reported.tsv
    invalidated.tsv


There's plenty of metadata, but it seems that the train-dev-test split has been predefined so lets use that.

[pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) makes it easy to read, filter, and manipulate metadata in tables.
Lets try to preprocess all metadata here so we don't have to worry about it later.

In [42]:
import pandas as pd
from IPython.display import display, Markdown


# Lexicographic order of labels as a fixed index target to label mapping
target2lang = tuple(sorted(languages))
lang2target = {lang: target for target, lang in enumerate(target2lang)}

print("lang2target:", lang2target)
print("target2lang:", target2lang)


def expand_metadata(row):
    """
    Update dataframe row by generating a unique utterance id,
    expanding the absolute path to the mp3 file,
    and adding an integer target for the label.
    """
    row.id = "{:s}_{:s}".format(
        row.path.split(".mp3", 1)[0].split("common_voice_", 1)[1],
        row.split)
    row.path = os.path.join(datadir, row.lang, "clips", row.path)
    row.target = lang2target[row.lang]
    return row


def tsv_to_lang_dataframe(lang, split):
    """
    Given a language and dataset split (train, dev, test),
    load the Common Voice metadata tsv-file from disk into a pandas.DataFrame.
    Preprocess all rows by dropping unneeded columns and adding new metadata.
    """
    df = pd.read_csv(
        os.path.join(datadir, lang, split + ".tsv"),
        sep='\t',
        # We only need these columns from the metadata
        usecols=("client_id", "path", "sentence"))
    # Add language label as column
    df.insert(len(df.columns), "lang", lang)
    # Add split name to every row for easier filtering
    df.insert(len(df.columns), "split", split)
    # Add placeholders for integer targets and utterance ids generated row-wise
    df.insert(len(df.columns), "target", -1)
    df.insert(len(df.columns), "id", "")
    # Create new metadata columns
    df = df.transform(expand_metadata, axis=1)
    return df


split_names = ("train", "dev", "test")

# Concatenate metadata for all 4 languages into a single table for each split
splits = [pd.concat([tsv_to_lang_dataframe(lang, split) for lang in target2lang])
          for split in split_names]

# Concatenate split metadata into a single table, indexed by utterance ids
meta = (pd.concat(splits)
        .set_index("id", drop=True, verify_integrity=True)
        .sort_index())
del splits

for split in split_names:
    display(Markdown("### " + split))
    display(meta[meta["split"]==split])

lang2target: {'fi': 0}
target2lang: ('fi',)


### train

,client_id,path,sentence,lang,split,target
id,,,,,,
fi_22986631_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Mitä nyt tekisimme?,fi,train,0
fi_22986632_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Äänestämme tämän vuoksi toisin kuin maataloude...,fi,train,0
fi_22986633_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,"Rupeatko remmiin, vai et?",fi,train,0
fi_22986634_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Äänestin näin ollen mietinnön puolesta.,fi,train,0
fi_22986635_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,"Kiitos, että tulitte ja opetitte meille viisau...",fi,train,0
...,...,...,...,...,...,...
fi_25900684_train,cea18d12298f2bbae520713a1cfcf05e63b2b9470dcc0a...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Jopa se puhaltaa.,fi,train,0
fi_25913820_train,cea18d12298f2bbae520713a1cfcf05e63b2b9470dcc0a...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Suomalainen on itsepäinen.,fi,train,0
fi_25941268_train,cea18d12298f2bbae520713a1cfcf05e63b2b9470dcc0a...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Amsterdam päätti kansallisten hallitusten epäo...,fi,train,0


### dev

,client_id,path,sentence,lang,split,target
id,,,,,,
fi_22994737_dev,f6c374d6e4e004c91278cc96d760776b435fb9983e35a7...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Äänestämme nyt kvestoreiden arvojärjestyksestä...,fi,dev,0
fi_22994739_dev,f6c374d6e4e004c91278cc96d760776b435fb9983e35a7...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Emme olleet vielä ehtineet toipua järkytyksest...,fi,dev,0
fi_22994740_dev,f6c374d6e4e004c91278cc96d760776b435fb9983e35a7...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,"Äänestin mietinnön puolesta, koska köyhyyden p...",fi,dev,0
fi_22994742_dev,f6c374d6e4e004c91278cc96d760776b435fb9983e35a7...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Yhä se huutaa.,fi,dev,0
fi_22994798_dev,f6c374d6e4e004c91278cc96d760776b435fb9983e35a7...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,"Äänestämme mietinnön puolesta ja toivomme, ett...",fi,dev,0
...,...,...,...,...,...,...
fi_25466549_dev,8a47121d35e6f8cd6ef71e56b2aca08664ca4ff3db6579...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Äänestämme myöhemmin tänään monista tällaisist...,fi,dev,0
fi_25466554_dev,8a47121d35e6f8cd6ef71e56b2aca08664ca4ff3db6579...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Tällin jälkeen mies ei ollut moksiskaan.,fi,dev,0
fi_25466556_dev,8a47121d35e6f8cd6ef71e56b2aca08664ca4ff3db6579...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,"Rikkaat, koko elämänsä työtä kartelleet, oliva...",fi,dev,0


### test

,client_id,path,sentence,lang,split,target
id,,,,,,
fi_22996506_test,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Silloin hänen pönkittynyt egonsa saisi hänet v...,fi,test,0
fi_22996507_test,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Hauki voi syödä suurenkin kalan.,fi,test,0
fi_22996553_test,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Kaupungissa ei häntä tarpeeksi tunneta puodeissa.,fi,test,0
fi_22996555_test,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Ville oli ihmeissään.,fi,test,0
fi_22996575_test,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Toivottavasti se juoksisi pakoon suolle päin j...,fi,test,0
...,...,...,...,...,...,...
fi_25812319_test,09d32b4e420e3e5231d0741310749d70ba5e9dcd71acde...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Kokoontumisrajoituksia on tarpeen edelleen ase...,fi,test,0
fi_25814463_test,09d32b4e420e3e5231d0741310749d70ba5e9dcd71acde...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Myös ulkoharrastuspaikat avataan kokoontumisra...,fi,test,0
fi_25838955_test,635303a7d1e2ea13318960e7d71fd39251278ed8c30a11...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Nyt en missään tapauksessa sinua auta.,fi,test,0


### Checking that all splits are disjoint by speaker

To ensure our neural network will learn what language is being spoken and not who is speaking, we want to test it on data that does not have any voices present in the training data.
The `client_id` should correspond to a unique, pseudonymized identifier for every speaker.

Lets check all splits are disjoint by speaker id.

In [43]:
def assert_splits_disjoint_by_speaker(meta):
    split2spk = {split: set(meta[meta["split"]==split].client_id.to_numpy())
                 for split in split_names}

    for split, spk in split2spk.items():
        print("split {} has {} speakers".format(split, len(spk)))

    print()
    print("asserting all are disjoint")
    assert split2spk["train"] & split2spk["test"] == set(), "train and test, mutual speakers"
    assert split2spk["train"] & split2spk["dev"]  == set(), "train and dev, mutual speakers"
    assert split2spk["dev"]   & split2spk["test"] == set(), "dev and test, mutual speakers"
    print("ok")


assert_splits_disjoint_by_speaker(meta)

split train has 3 speakers
split dev has 10 speakers
split test has 91 speakers

asserting all are disjoint
ok


We can see that none of the speakers are in two or more dataset splits.
We also see that the test set has a lot of unique speakers who are not in the training set.
This is good because we want to test that our neural network classifier knows how to classify input from unknown speakers.

### Checking that all audio files exist

In [44]:
for uttid, row in meta.iterrows():
    assert os.path.exists(row["path"]), row["path"] + " does not exist"
print("ok")

ok


## Balancing the language distribution

Lets see how many samples we have per language.

In [54]:
import seaborn as sns


# sns.set(rc={'figure.figsize': (8, 6)})
# ax = sns.countplot(
#     x="split",
#     order=split_names,
#     hue="lang",
#     hue_order=target2lang,
#     data=meta)
# ax.set_title("Total amount of audio samples")
# plt.show()

We can see that the amount of samples with Mongolian, Tamil, and Turkish speech are quite balanced, but we have significantly larger amounts of Estonian speech.
More data is of course always better, but if there is too much of one label compared to the others, our neural network might overfit on this label.

But these are only the counts of audio files, how much speech do we have in total per language?
We need to read every file to get a reliable answer.
See also [SoX](http://sox.sourceforge.net/Main/HomePage) for a good command line tool.

In [46]:
import miniaudio


meta["duration"] = np.array([
    miniaudio.mp3_get_file_info(path).duration for path in meta.path], np.float32)
meta

,client_id,path,sentence,lang,split,target,duration
id,,,,,,,
fi_22986631_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Mitä nyt tekisimme?,fi,train,0,4.776
fi_22986632_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Äänestämme tämän vuoksi toisin kuin maataloude...,fi,train,0,6.264
fi_22986633_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,"Rupeatko remmiin, vai et?",fi,train,0,3.504
fi_22986634_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Äänestin näin ollen mietinnön puolesta.,fi,train,0,4.344
fi_22986635_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,"Kiitos, että tulitte ja opetitte meille viisau...",fi,train,0,5.616
...,...,...,...,...,...,...,...
fi_25913820_train,cea18d12298f2bbae520713a1cfcf05e63b2b9470dcc0a...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Suomalainen on itsepäinen.,fi,train,0,3.564
fi_25938262_test,09d32b4e420e3e5231d0741310749d70ba5e9dcd71acde...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Miksi ne ajoivat?,fi,test,0,1.584
fi_25941268_train,cea18d12298f2bbae520713a1cfcf05e63b2b9470dcc0a...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Amsterdam päätti kansallisten hallitusten epäo...,fi,train,0,5.076


In [53]:
def plot_duration_distribution(data):
    sns.set(rc={'figure.figsize': (8, 6)})
    
    ax = sns.boxplot(
        x="split",
        order=split_names,
        y="duration",
        hue="lang",
        hue_order=target2lang,
        data=data)
    ax.set_title("Median audio file duration in seconds")
    plt.show()

    ax = sns.barplot(
        x="split",
        order=split_names,
        y="duration",
        hue="lang",
        hue_order=target2lang,
        data=data,
        ci=None,
        estimator=np.sum)
    ax.set_title("Total amount of audio in seconds")
    plt.show()


#plot_duration_distribution(meta)

The median length of Estonian samples is approx. 2.5 seconds greater compared to Turkish samples, which have the shortest median length.
We can also see that the total amount of Estonian speech is much larger compared to other languages in our datasets.
Notice also the significant amount of outliers with long durations in the Tamil and Turkish datasets.

Lets do simple random oversampling for the training split using this approach:

1. Select the target language according to maximum total amount of speech in seconds (Estonian).
2. Compute differences in total durations between the target language and the three other languages.
3. Compute median signal length by language.
4. Compute sample sizes by dividing the duration deltas with median signal lengths, separately for each language.
5. Draw samples with replacement from the metadata separately for each language.
6. Merge samples with rest of the metadata and verify there are no duplicate ids.

In [55]:
def random_oversampling(meta):
    groupby_lang = meta[["lang", "duration"]].groupby("lang")
    
    total_dur = groupby_lang.sum()
    target_lang = total_dur.idxmax()[0]
    print("target lang:", target_lang)
    print("total durations:")
    display(total_dur)
    
    total_dur_delta = total_dur.loc[target_lang] - total_dur
    print("total duration delta to target lang:")
    display(total_dur_delta)
    
    median_dur = groupby_lang.median()
    print("median durations:")
    display(median_dur)
    
    sample_sizes = (total_dur_delta / median_dur).astype(np.int32)
    print("median duration weighted sample sizes based on total duration differences:")
    display(sample_sizes)
    
    samples = []
    
    for lang in groupby_lang.groups:
        sample_size = sample_sizes.loc[lang][0]
        sample = (meta[meta["lang"]==lang]
                  .sample(n=sample_size, replace=True, random_state=np_rng.bit_generator)
                  .reset_index()
                  .transform(update_sample_id, axis=1))
        samples.append(sample)

    return pd.concat(samples).set_index("id", drop=True, verify_integrity=True)


def update_sample_id(row):
    row["id"] = "{}_copy_{}".format(row["id"], row.name)
    return row

    
# Augment training set metadata
meta = pd.concat([random_oversampling(meta[meta["split"]=="train"]), meta]).sort_index()

assert not meta.isna().any(axis=None), "NaNs in metadata after augmentation"
#plot_duration_distribution(meta)
assert_splits_disjoint_by_speaker(meta)
meta

target lang: fi
total durations:


,duration
lang,
fi,9757.007812


total duration delta to target lang:


,duration
lang,
fi,0.0


median durations:


,duration
lang,
fi,4.212


median duration weighted sample sizes based on total duration differences:


,duration
lang,
fi,0


split train has 3 speakers
split dev has 10 speakers
split test has 91 speakers

asserting all are disjoint
ok


,client_id,path,sentence,lang,split,target,duration
id,,,,,,,
fi_22986631_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Mitä nyt tekisimme?,fi,train,0,4.776
fi_22986632_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Äänestämme tämän vuoksi toisin kuin maataloude...,fi,train,0,6.264
fi_22986633_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,"Rupeatko remmiin, vai et?",fi,train,0,3.504
fi_22986634_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Äänestin näin ollen mietinnön puolesta.,fi,train,0,4.344
fi_22986635_train,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,"Kiitos, että tulitte ja opetitte meille viisau...",fi,train,0,5.616
...,...,...,...,...,...,...,...
fi_25913820_train,cea18d12298f2bbae520713a1cfcf05e63b2b9470dcc0a...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Suomalainen on itsepäinen.,fi,train,0,3.564
fi_25938262_test,09d32b4e420e3e5231d0741310749d70ba5e9dcd71acde...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Miksi ne ajoivat?,fi,test,0,1.584
fi_25941268_train,cea18d12298f2bbae520713a1cfcf05e63b2b9470dcc0a...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Amsterdam päätti kansallisten hallitusten epäo...,fi,train,0,5.076


Speech data augmentation is a common research topic.
There are [better](https://www.isca-speech.org/archive/interspeech_2015/papers/i15_3586.pdf) ways to augment data than the simple duplication of metadata rows we did here.
One approach (which we won't be doing here) which is easy to implement and might work well is to take copies of signals and make them randomly a bit faster or slower.
For example, draw randomly speed ratios from `[0.9, 1.1]` and resample the signal by multiplying its sample rate with the random ratio.

## Inspecting the audio

Lets take a look at the speech data and listen to a few randomly picked samples from each label.
We pick 2 random samples for each language from the training set.

In [49]:
samples = (meta[meta["split"]=="train"]
           .groupby("lang")
           .sample(n=2, random_state=np_rng.bit_generator))
samples

,client_id,path,sentence,lang,split,target,duration
id,,,,,,,
fi_25263300_train,31926f462f259114ba8596c4814ed1df0e43021720bc7b...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,"Ei, hiljaa oli Topi ystäväänsä huudellut.",fi,train,0,4.320
fi_25335195_train,31926f462f259114ba8596c4814ed1df0e43021720bc7b...,/Users/haily/Downloads/cv-corpus-7.0-2021-07-2...,Oranssin ympyrän reunasta reunaan meni musta k...,fi,train,0,4.788


Then lets read the mp3-files from disk, plot the signals, and listen to the audio.

In [56]:
from IPython.display import display, Audio, HTML
import scipy.signal


def read_mp3(path, resample_rate=16000):
    if isinstance(path, bytes):
        # If path is a tf.string tensor, it will be in bytes
        path = path.decode("utf-8")
        
    f = miniaudio.mp3_read_file_f32(path)
    
    # Downsample to target rate, 16 kHz is commonly used for speech data
    new_len = round(len(f.samples) * float(resample_rate) / f.sample_rate)
    signal = scipy.signal.resample(f.samples, new_len)
    
    # Normalize to [-1, 1]
    signal /= np.abs(signal).max()
    
    return signal, resample_rate


def embed_audio(signal, rate):
    display(Audio(data=signal, rate=rate, embed=True, normalize=False))

    
def plot_signal(data, figsize=(6, 0.5), **kwargs):
    ax = sns.lineplot(data=data, lw=0.1, **kwargs)
    ax.set_axis_off()
    ax.margins(0)
    plt.gcf().set_size_inches(*figsize)
    plt.show()

    
def plot_separator():
    display(HTML(data="<hr style='border: 2px solid'>"))

    
# for sentence, lang, clip_path in samples[["sentence", "lang", "path"]].to_numpy():
#     signal, rate = read_mp3(clip_path)
#     plot_signal(signal)
#     print("length: {} sec".format(signal.size / rate))
#     print("lang:", lang)
#     print("sentence:", sentence)
#     embed_audio(signal, rate)
#     plot_separator()

One of the most challenging aspects of the Mozilla Common Voice dataset is that the audio quality varies greatly: different microphones, background noise, user is speaking close to the device or far away etc.
It is difficult to ensure that a neural network will learn to classify different languages as opposed to classifying distinct acoustic artefacts from specific microphones.
There's a [vast amount of research](https://www.isca-speech.org/archive/Interspeech_2020/) being done on developing techniques for solving these kind of problems.
However, these are well out of scope for this simple example and we won't be studying them here.


## Spectral representations

It is usually not possible (at least not yet in 2020) to detect languages directly from the waveform.
Instead, the [fast Fourier transform](https://en.wikipedia.org/wiki/Short-time_Fourier_transform) (FFT) is applied on small, overlapping windows of the signal to get a 2-dimensional representation of energies in different frequency bands.
See [this](https://wiki.aalto.fi/display/ITSP/Spectrogram+and+the+STFT) for further details.

However, output from the FFT is usually not usable directly and must be refined.
Lets begin by selecting the first signal from our random sample and extract the power spectrogram.

### Power spectrogram

In [57]:
from lidbox.features.audio import spectrograms


def plot_spectrogram(S, cmap="viridis", figsize=None, **kwargs):
    if figsize is None:
        figsize = S.shape[0]/50, S.shape[1]/50
    ax = sns.heatmap(S.T, cbar=False, cmap=cmap, **kwargs)
    ax.invert_yaxis()
    ax.set_axis_off()
    ax.margins(0)
    plt.gcf().set_size_inches(*figsize)
    plt.show()

    
sample = samples[["sentence", "lang", "path"]].to_numpy()[0]
sentence, lang, clip_path = sample

signal, rate = read_mp3(clip_path)
# plot_signal(signal)

powspec = spectrograms([signal], rate)[0]

# plot_spectrogram(powspec.numpy())

This representation is very sparse, with zeros everywhere except in the lowest frequency bands.
The main problem here is that relative differences between energy values are very large, making it different to compare large changes in energy.
These differences can be reduced by mapping the values onto a logarithmic scale.

The [decibel-scale](https://en.wikipedia.org/wiki/Decibel) is a common choice.
We will use the maximum value of `powspec` as the reference power ($\text{P}_0$).

### Decibel-scale spectrogram

16000

In [58]:
from lidbox.features.audio import power_to_db


#dbspec = power_to_db([powspec])[0]
#plot_spectrogram(dbspec.numpy())

This is an improvement, but the representation is still rather sparse.
We also see that most speech information is in the lower bands, with a bit of energy in the higher frequencies.
A common approach is to "squeeze together" the y-axis of all frequency bands by using a different scale, such as the [Mel-scale](https://en.wikipedia.org/wiki/Mel_scale).
Lets "squeeze" the current 256 frequency bins into 40 Mel-bins.

### Log-scale Mel-spectrogram

**Note** that we are scaling different things here.
The Mel-scale warps the frequency bins (y-axis), while the logarithm is used to reduce relative differences between individual spectrogram values (pixels).

In [61]:
from lidbox.features.audio import linear_to_mel


def logmelspectrograms(signals, rate):
    powspecs = spectrograms(signals, rate)
    melspecs = linear_to_mel(powspecs, rate, num_mel_bins=40)
    return tf.math.log(melspecs + 1e-6)
    

logmelspec = logmelspectrograms([signal], rate)[0]
#plot_spectrogram(logmelspec.numpy())

One common normalization technique is frequency channel standardization, i.e. normalization of rows to zero mean and unit variance.

In [ ]:
from lidbox.features import cmvn

logmelspec_mv = cmvn([logmelspec])[0]
# plot_spectrogram(logmelspec_mv.numpy())

In [63]:
logmelspectrograms([signal], rate).shape

TensorShape([1, 430, 40])

In [64]:
cmvn([logmelspec]).shape

TensorShape([1, 430, 40])

Or only mean-normalization if you think the variances contain important information.

In [ ]:
logmelspec_m = cmvn([logmelspec], normalize_variance=False)[0]
plot_spectrogram(logmelspec_m.numpy())

## Cepstral representations

Another common representation are the Mel-frequency cepstral coefficients (MFCC), which are obtained by applying the [discrete cosine transform](https://en.wikipedia.org/wiki/Discrete_cosine_transform) on the log-scale Mel-spectrogram.

### MFCC

In [67]:
def plot_cepstra(X, figsize=None):
    if not figsize:
        figsize = (X.shape[0]/50, X.shape[1]/20)
    plot_spectrogram(X, cmap="RdBu_r", figsize=figsize)

    
mfcc = tf.signal.mfccs_from_log_mel_spectrograms([logmelspec])[0]
#plot_cepstra(mfcc.numpy())

Most of the information is concentrated in the lower coefficients.
It is common to drop the 0th coefficient and select a subset starting at 1, e.g. 1 to 20.
See [this post](http://practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/) for more details.

In [68]:
mfcc = mfcc[:,1:21]
# plot_cepstra(mfcc.numpy())

TensorShape([430, 20])

Now we have a very compact representation, but most of the variance is still in the lower coefficients and overshadows the smaller changes in higher coefficients.
We can normalize the MFCC matrix row-wise by standardizing each row to zero mean and unit variance.
This is commonly called cepstral mean and variance normalization (CMVN).

### MFCC + CMVN

In [ ]:
mfcc_cmvn = cmvn([mfcc])[0]
# plot_cepstra(mfcc_cmvn.numpy())

### Which one is best?

Speech feature extraction is a large, active research topic and it is impossible to choose one representation that would work well in all situations.
Common choices in state-of-the-art spoken language identification are log-scale Mel-spectrograms and MFCCs, with different normalization approaches.
For example, [here](https://github.com/swshon/dialectID_e2e) is an experiment in Arabic dialect identification, where log-scale Mel-spectra (referred to as FBANK) produced slightly better results compared to MFCCs.

It is not obvious when to choose which representation, or if we should even use the FFT at all.
You can read [this post](https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html) for a more detailed discussion.

## Voice activity detection

It is common for speech datasets to contain audio samples with short segments of silence or sounds that are not speech.
Since these are usually irrelevant for making a language classification decision, we would prefer to discard such segments.
This is called voice activity detection (VAD) and it is another large, active research area.
[Here](https://wiki.aalto.fi/pages/viewpage.action?pageId=151500905) is a brief overview of VAD. 

Non-speech segments can be either noise or silence. 
Separating non-speech noise from speech is non-trivial but possible, for example with [neural networks](https://www.isca-speech.org/archive/Interspeech_2019/pdfs/1354.pdf).
Silence, on the other hand, shows up as zeros in our speech representations, since these segments contain lower energy values compared to segments with speech.
Such non-speech segments are therefore easy to detect and discard, for example by comparing the energy of the segment to the average energy of the whole sample.

If the samples in our example do not contain much background noise, a simple energy-based VAD technique should be enough to drop all silent segments.
We'll use the [root mean square](https://en.wikipedia.org/wiki/Root_mean_square) (RMS) energy to detect short silence segments.
`lidbox` has a simple energy-based VAD function, which we will use as follows:

1. Divide the signal into non-overlapping 10 ms long windows.
2. Compute RMS of each window.
3. Reduce all window RMS values by averaging to get a single mean RMS value.
4. Set a decision threshold at 0.1 for marking silence windows. In other words, if the window RMS is less than 0.1 of the mean RMS, mark the window as silence.


In [71]:
from lidbox.features.audio import framewise_rms_energy_vad_decisions
import matplotlib.patches as patches


sentence, lang, clip_path = sample
signal, rate = read_mp3(clip_path)

window_ms = tf.constant(10, tf.int32)
window_frame_length = (window_ms * rate) // 1000

# Get binary VAD decisions for each 10 ms window
vad_1 = framewise_rms_energy_vad_decisions(
    signal=signal,
    sample_rate=rate,
    frame_step_ms=window_ms,
    strength=0.1)

# # Plot unfiltered signal
# sns.set(rc={'figure.figsize': (6, 0.5)})
# ax = sns.lineplot(data=signal, lw=0.1, legend=None)
# ax.set_axis_off()
# ax.margins(0)

# # Plot shaded area over samples marked as not speech (VAD == 0)
# for x, is_speech in enumerate(vad_1.numpy()):
#     if not is_speech:
#         rect = patches.Rectangle(
#             (x*window_frame_length, -1),
#             window_frame_length,
#             2,
#             linewidth=0,
#             color='gray',
#             alpha=0.2)
#         ax.add_patch(rect)
# plt.show()

print("lang:", lang)
print("sentence: '{}'".format(sentence))
#embed_audio(signal, rate)

# Partition the signal into 10 ms windows to match the VAD decisions
windows = tf.signal.frame(signal, window_frame_length, window_frame_length)
# Filter signal with VAD decision == 1 (remove gray areas)
filtered_signal = tf.reshape(windows[vad_1], [-1])

# plot_signal(filtered_signal)
print("dropped {:d} out of {:d} frames, leaving {:.3f} of the original signal".format(
    signal.shape[0] - filtered_signal.shape[0],
    signal.shape[0],
    filtered_signal.shape[0]/signal.shape[0]))
#embed_audio(filtered_signal, rate)

lang: fi
sentence: 'Ei, hiljaa oli Topi ystäväänsä huudellut.'
dropped 27200 out of 69120 frames, leaving 0.606 of the original signal


In [75]:
filtered_signal.shape
windows.shape


TensorShape([432, 160])

The filtered signal has less silence, but some of the pauses between words sound too short and unnatural.
We would prefer not to remove small pauses that normally occur between words, so lets say all pauses shorter than 300 ms should not be filtered out.
Lets also move all VAD code into a function.

In [78]:
def remove_silence(signal, rate):
    window_ms = tf.constant(10, tf.int32)
    window_frames = (window_ms * rate) // 1000
    
    # Get binary VAD decisions for each 10 ms window
    vad_1 = framewise_rms_energy_vad_decisions(
        signal=signal,
        sample_rate=rate,
        frame_step_ms=window_ms,
        # Do not return VAD = 0 decisions for sequences shorter than 300 ms
        min_non_speech_ms=300,
        strength=0.1)
    
    # Partition the signal into 10 ms windows to match the VAD decisions
    windows = tf.signal.frame(signal, window_frames, window_frames)
    # Filter signal with VAD decision == 1
    return tf.reshape(windows[vad_1], [-1])


sentence, lang, clip_path = sample
signal, rate = read_mp3(clip_path)

filtered_signal = remove_silence(signal, rate)
# plot_signal(filtered_signal)

print("dropped {:d} out of {:d} frames, leaving {:.3f} of the original signal".format(
    signal.shape[0] - filtered_signal.shape[0],
    signal.shape[0],
    filtered_signal.shape[0]/signal.shape[0]))

print("lang:", lang)
print("sentence: '{}'".format(sentence))
#embed_audio(filtered_signal, rate)

dropped 24640 out of 69120 frames, leaving 0.644 of the original signal
lang: fi
sentence: 'Ei, hiljaa oli Topi ystäväänsä huudellut.'


We dropped some silence segments but left most of the speech intact, perhaps this is enough for our example.

Although this VAD approach is simple and works ok for our data, it will not work for speech data with non-speech sounds in the background like music or noise.
For such data we might need more powerful VAD filters such as neural networks that have been trained on a speech vs non-speech classification task with large amounts of different noise.

But lets not add more complexity to our example.
We'll use the RMS based filter for all other signals too.

## Comparison of representations

Lets extract these features for all signals in our random sample.

In [81]:
for sentence, lang, clip_path in samples[["sentence", "lang", "path"]].to_numpy():
    signal_before_vad, rate = read_mp3(clip_path)
    signal = remove_silence(signal_before_vad, rate)
    
    logmelspec = logmelspectrograms([signal], rate)[0]
    #logmelspec_mvn = cmvn([logmelspec], normalize_variance=False)[0]
    logmelspec_mvn = cmvn([logmelspec])[0]
    
    mfcc = tf.signal.mfccs_from_log_mel_spectrograms([logmelspec])[0]
    mfcc = mfcc[:,1:21]
    mfcc_cmvn = cmvn([mfcc])[0]
    
    plot_width = logmelspec.shape[0]/50
    #plot_signal(signal.numpy(), figsize=(plot_width, .6))
    print("VAD: {} -> {} sec".format(
        signal_before_vad.size / rate,
        signal.numpy().size / rate))
    print("lang:", lang)
    print("sentence:", sentence)
    #embed_audio(signal.numpy(), rate)
    
    #plot_spectrogram(logmelspec_mvn.numpy(), figsize=(plot_width, 1.2))
    #plot_cepstra(mfcc_cmvn.numpy(), figsize=(plot_width, .6))
    
    #plot_separator()

VAD: 4.32 -> 2.78 sec
lang: fi
sentence: Ei, hiljaa oli Topi ystäväänsä huudellut.
VAD: 4.788 -> 3.9 sec
lang: fi
sentence: Oranssin ympyrän reunasta reunaan meni musta katkoviiva.


## Loading the samples to a `tf.data.Dataset` iterator

Our dataset is relatively small (2.5 GiB) and we might be able to read all files into signals and keep them in main memory.
However, most speech datasets are much larger due to the amount of data needed for training neural network models that would be of any practical use.
We need some kind of lazy iteration or streaming solution that views only one part of the dataset at a time.
One such solution is to represent the dataset as a [TensorFlow iterator](https://www.tensorflow.org/api_docs/python/tf/data/Dataset), which evaluates its contents only when they are needed, similar to the [MapReduce](https://en.wikipedia.org/wiki/MapReduce) programming model for big data.

The downside with lazy iteration or streaming is that we lose the capability of doing random access by row id.
However, this shouldn't be a problem since we can always keep the whole metadata table in memory and do random access on its rows whenever needed.

Another benefit of TensorFlow dataset iterators is that we can map arbitrary [`tf.function`](https://www.tensorflow.org/api_docs/python/tf/function)s over the dataset and TensorFlow will automatically parallelize the computations and place them on different devices, such as the GPU.
The core architecture of `lidbox` has been organized around the `tf.data.Dataset` API, leaving all the heavy lifting for TensorFlow to handle.

But before we load all our speech data, lets warmup with our small random sample of 8 rows.

In [82]:
len(samples)

2

Lets load it into a `tf.data.Dataset`.

In [83]:
def metadata_to_dataset_input(meta):   
    # Create a mapping from column names to all values under the column as tensors
    return {
        "id": tf.constant(meta.index, tf.string),
        "path": tf.constant(meta.path, tf.string),
        "lang": tf.constant(meta.lang, tf.string),
        "target": tf.constant(meta.target, tf.int32),
        "split": tf.constant(meta.split, tf.string),
    }


sample_ds = tf.data.Dataset.from_tensor_slices(metadata_to_dataset_input(samples))
sample_ds

<TensorSliceDataset shapes: {id: (), path: (), lang: (), target: (), split: ()}, types: {id: tf.string, path: tf.string, lang: tf.string, target: tf.int32, split: tf.string}>

All elements produced by the `Dataset` iterator are `dict`s of (string, Tensor) pairs, where the string denotes the metadata type.

Although the `Dataset` object is primarily for automating large-scale data processing pipelines, it is easy to extract all elements as `numpy`-values:

In [84]:
for x in sample_ds.as_numpy_iterator():
    display(x)

{'id': b'fi_25263300_train',
 'path': b'/Users/haily/Downloads/cv-corpus-7.0-2021-07-21/fi/clips/common_voice_fi_25263300.mp3',
 'lang': b'fi',
 'target': 0,
 'split': b'train'}

{'id': b'fi_25335195_train',
 'path': b'/Users/haily/Downloads/cv-corpus-7.0-2021-07-21/fi/clips/common_voice_fi_25335195.mp3',
 'lang': b'fi',
 'target': 0,
 'split': b'train'}

### Reading audio files

Lets load the signals by [mapping](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) a file reading function for each element over the whole dataset.
We'll add a `tf.data.Dataset` function wrapper on top of `read_mp3`, which we defined earlier.
TensorFlow will infer the input and output values of the wrapper as tensors from the type signature of dataset elements.
We must use `tf.numpy_function` if we want to allow calling the non-TensorFlow function `read_mp3` also from
inside the graph environment.
It might not be as efficient as using TensorFlow ops but reading a file would have a lot of latency anyway so this is not such a big hit for performance.
Besides, we can always hide the latency by reading several files in parallel.

In [85]:
def read_mp3_wrapper(x):
    signal, sample_rate = tf.numpy_function(
        # Function
        read_mp3,
        # Argument list
        [x["path"]],
        # Return value types
        [tf.float32, tf.int64])
    return dict(x, signal=signal, sample_rate=tf.cast(sample_rate, tf.int32))


for x in sample_ds.map(read_mp3_wrapper).as_numpy_iterator():
    print("id: {}".format(x["id"].decode("utf-8")))
    print("signal.shape: {}, sample rate: {}".format(x["signal"].shape, x["sample_rate"]))
    print()

id: fi_25263300_train
signal.shape: (69120,), sample rate: 16000

id: fi_25335195_train
signal.shape: (76608,), sample rate: 16000



### Removing silence and extracting features

Organizing all preprocessing steps as functions that can be mapped over the `Dataset` object allows us to represent complex transformations easily.

In [87]:
def remove_silence_wrapper(x):
    return dict(x, signal=remove_silence(x["signal"], x["sample_rate"]))


def batch_extract_features(x):
    with tf.device("GPU"):
        signals, rates = x["signal"], x["sample_rate"]
        logmelspecs = logmelspectrograms(signals, rates[0])
        #logmelspecs_smn = cmvn(logmelspecs, normalize_variance=False)
        logmelspecs_smn = cmvn(logmelspecs)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(logmelspecs)
        mfccs = mfccs[...,1:21]
        mfccs_cmvn = cmvn(mfccs)
    return dict(x, logmelspec=logmelspecs_smn, mfcc=mfccs_cmvn)


features_ds = (sample_ds.map(read_mp3_wrapper)
                 .map(remove_silence_wrapper)
                 .batch(1)
                 .map(batch_extract_features)
                 .unbatch())

for x in features_ds.as_numpy_iterator():
    print(x["id"])
    for k in ("signal", "logmelspec", "mfcc"):
        print("{}.shape: {}".format(k, x[k].shape))
    print()

b'fi_25263300_train'
signal.shape: (44480,)
logmelspec.shape: (276, 40)
mfcc.shape: (276, 20)

b'fi_25335195_train'
signal.shape: (62400,)
logmelspec.shape: (388, 40)
mfcc.shape: (388, 20)



### Inspecting dataset contents in TensorBoard

`lidbox` has a helper function for dumping element information into [`TensorBoard`](https://www.tensorflow.org/tensorboard) summaries.
This converts all 2D features into images, writes signals as audio summaries, and extracts utterance ids.

In [ ]:
import lidbox.data.steps as ds_steps


cachedir = os.path.join(workdir, "cache")

_ = ds_steps.consume_to_tensorboard(
    # Rename logmelspec as 'input', these will be plotted as images
    ds=features_ds.map(lambda x: dict(x, input=x["logmelspec"])),
    summary_dir=os.path.join(cachedir, "tensorboard", "data", "sample"),
    config={"batch_size": 1, "image_size_multiplier": 4})

Open a terminal and launch TensorBoard to view the summaries written to `$wrkdir/cache/tensorboard/dataset/sample`:
```
tensorboard --logdir /data/exp/cv4/cache/tensorboard
```
Then open the url in a browser and inspect the contents.
You can leave the server running, since we'll log the training progress to the same directory.

## Loading all data

We'll now begin loading everything from disk and preparing a pipeline from mp3-filepaths to neural network input.
We'll use the autotune feature of `tf.data` to allow TensorFlow figure out automatically how much of the pipeline should be split up into parallel calls.

In [ ]:
import lidbox.data.steps as ds_steps

TF_AUTOTUNE = tf.data.experimental.AUTOTUNE


def signal_is_not_empty(x):
    return tf.size(x["signal"]) > 0
    

def pipeline_from_metadata(data, shuffle=False):
    if shuffle:
        # Shuffle metadata to get an even distribution of labels
        data = data.sample(frac=1, random_state=np_rng.bit_generator)
    ds = (
        # Initialize dataset from metadata
        tf.data.Dataset.from_tensor_slices(metadata_to_dataset_input(data))
        # Read mp3 files from disk in parallel
        .map(read_mp3_wrapper, num_parallel_calls=TF_AUTOTUNE)
        # Apply RMS VAD to drop silence from all signals
        .map(remove_silence_wrapper, num_parallel_calls=TF_AUTOTUNE)
        # Drop signals that VAD removed completely
        .filter(signal_is_not_empty)
        # Extract features in parallel
        .batch(1)
        .map(batch_extract_features, num_parallel_calls=TF_AUTOTUNE)
        .unbatch()
    )
    return ds


# Mapping from dataset split names to tf.data.Dataset objects
split2ds = {
    split: pipeline_from_metadata(meta[meta["split"]==split], shuffle=split=="train")
    for split in split_names
}

### Testing pipeline performance

Note that we only constructed the pipeline with all steps we want to compute.
All TensorFlow ops are computed only when elements are requested from the iterator.

Lets iterate over the training dataset from first to last element to ensure the pipeline will not be a performance bottleneck during training.

In [ ]:
_ = ds_steps.consume(split2ds["train"], log_interval=2000)

### Caching pipeline state

We can [cache](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#cache) the iterator state as a single binary file at arbitrary stages.
This allows us to automatically skip all steps that precede the call to `tf.Dataset.cache`.

Lets cache the training dataset and iterate again over all elements to fill the cache.
**Note** that you will still be storing all data on the disk (4.6 GiB new data), so this optimization is a space-time tradeoff.

In [ ]:
os.makedirs(os.path.join(cachedir, "data"))

split2ds["train"] = split2ds["train"].cache(os.path.join(cachedir, "data", "train"))
_ = ds_steps.consume(split2ds["train"], log_interval=2000)

If we iterate over the dataset again, TensorFlow should read all elements from the cache file.

In [ ]:
_ = ds_steps.consume(split2ds["train"], log_interval=2000)

As a side note, if your training environment has fast read-write access to a file system configured for reading and writing very large files, this optimization can be a very significant performance improvement.

**Note** also that all usual problems related to cache invalidation apply.
When caching extracted features and metadata to disk, be extra careful in your experiments to ensure you are not interpreting results computed on data from some outdated cache.

### Dumping a few batches to TensorBoard 

Lets extract 100 first elements of every split to TensorBoard.

In [ ]:
for split, ds in split2ds.items():
    _ = ds_steps.consume_to_tensorboard(
            ds.map(lambda x: dict(x, input=x["logmelspec"])),
            os.path.join(cachedir, "tensorboard", "data", split),
            {"batch_size": 1,
             "image_size_multiplier": 2,
             "num_batches": 100},
            exist_ok=True)

## Training a supervised, neural network language classifier

We have now configured an efficient data pipeline and extracted some data samples to summary files for TensorBoard.
It is time to train a classifier on the data.

### Drop metadata from dataset

During training, we only need a tuple of model input and targets.
We can therefore drop everything else from the dataset elements just before training starts.
This is also a good place to decide if we want to train on MFCCs or Mel-spectra.

In [ ]:
model_input_type = "logmelspec"

def as_model_input(x):
    return x[model_input_type], x["target"]


train_ds_demo = list(split2ds["train"]
                     .map(as_model_input)
                     .shuffle(100)
                     .take(6)
                     .as_numpy_iterator())

for input, target in train_ds_demo:
    print(input.shape, target2lang[target])
    if model_input_type == "mfcc":
        plot_cepstra(input)
    else:
        plot_spectrogram(input)
    plot_separator()

### Asserting all input is valid

Since the training dataset is cached, we can quickly iterate over all elements and check that we don't have any NaNs or negative targets.

In [ ]:
def assert_finite(x, y):
    tf.debugging.assert_all_finite(x, "non-finite input")
    tf.debugging.assert_non_negative(y, "negative target")
    return x, y

_ = ds_steps.consume(split2ds["train"].map(as_model_input).map(assert_finite), log_interval=5000)

It is also easy to compute stats on the dataset elements.
For example finding global minimum and maximum values of the inputs.

In [ ]:
x_min = split2ds["train"].map(as_model_input).reduce(
    tf.float32.max,
    lambda acc, elem: tf.math.minimum(acc, tf.math.reduce_min(elem[0])))

x_max = split2ds["train"].map(as_model_input).reduce(
    tf.float32.min,
    lambda acc, elem: tf.math.maximum(acc, tf.math.reduce_max(elem[0])))

print("input tensor global minimum: {}, maximum: {}".format(x_min.numpy(), x_max.numpy()))

### Selecting a model architecture

`lidbox` provides a small set of neural network model architectures out of the box.
Many of these architectures have good results in the literature for different datasets.
These models have been implemented in Keras, so you could replace the model we are using here with anything you want.

The ["x-vector"](http://danielpovey.com/files/2018_odyssey_xvector_lid.pdf) architecture has worked well in speaker and language identification so lets create an untrained Keras x-vector model.
One of its core features is learning fixed length vector representations (x-vectors) for input of arbitrary length.
These vectors are extracted from the first fully connected layer (`segment1`), without activation.
This opens up opportunities for doing all kinds of statistical analysis on these vectors, but that's out of scope for our example.

We'll try to regularize the network by adding frequency [channel dropout](https://dl.acm.org/doi/abs/10.1016/j.patrec.2017.09.023) with probability 0.8.
In other words, during training we set input rows randomly to zeros with probability 0.8.
This might avoid overfitting the network on frequency channels containing noise that is irrelevant for deciding the language.

In [ ]:
import lidbox.models.xvector as xvector


def create_model(num_freq_bins, num_labels):
    model = xvector.create([None, num_freq_bins], num_labels, channel_dropout_rate=0.8)
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5))
    return model


model = create_model(
    num_freq_bins=20 if model_input_type == "mfcc" else 40,
    num_labels=len(target2lang))
model.summary()

### Channel dropout demo

Here's what happens to the input during training.

In [ ]:
channel_dropout = tf.keras.layers.SpatialDropout1D(model.get_layer("channel_dropout").rate)

for input, target in train_ds_demo:
    print(input.shape, target2lang[target])
    input = channel_dropout(tf.expand_dims(input, 0), training=True)[0].numpy()
    if model_input_type == "mfcc":
        plot_cepstra(input)
    else:
        plot_spectrogram(input)
    plot_separator()

### Training the classifier

The validation set is needed after every epoch, so we might as well cache it.
**Note** that this writes 2.5 GiB of additional data to disk the first time the validation set is iterated over, i.e. at the end of epoch 1.
Also, we can't use batches since our input is of different lengths (perhaps with [ragged tensors](https://www.tensorflow.org/versions/r2.3/api_docs/python/tf/data/experimental/dense_to_ragged_batch)).

In [ ]:
callbacks = [
    # Write scalar metrics and network weights to TensorBoard
    tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(cachedir, "tensorboard", model.name),
        update_freq="epoch",
        write_images=True,
        profile_batch=0,
    ),
    # Stop training if validation loss has not improved from the global minimum in 10 epochs
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
    ),
    # Write model weights to cache everytime we get a new global minimum loss value
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(cachedir, "model", model.name),
        monitor='val_loss',
        save_weights_only=True,
        save_best_only=True,
        verbose=1,
    ),
]

train_ds = split2ds["train"].map(as_model_input).shuffle(1000)
dev_ds = split2ds["dev"].cache(os.path.join(cachedir, "data", "dev")).map(as_model_input)

# history = model.fit(
#     train_ds.batch(1),
#     validation_data=dev_ds.batch(1),
#     callbacks=callbacks,
#     verbose=2,
#     epochs=100)

## Evaluating the classifier


Lets run all test set samples through our trained model by loading the best weights from the cache.

In [ ]:
# from lidbox.util import predict_with_model


# test_ds = split2ds["test"].map(lambda x: dict(x, input=x["logmelspec"])).batch(1)

# _ = model.load_weights(os.path.join(cachedir, "model", model.name))
# utt2pred = predict_with_model(model, test_ds)

# test_meta = meta[meta["split"]=="test"]
# assert not test_meta.join(utt2pred).isna().any(axis=None), "missing predictions"
# test_meta = test_meta.join(utt2pred)
# test_meta

### Average detection cost ($\text{C}_\text{avg}$)

The de facto standard metric for evaluating spoken language classifiers might be the *average detection cost* ($\text{C}_\text{avg}$), which has been refined to its current form during past [language recognition competitions](https://tsapps.nist.gov/publication/get_pdf.cfm?pub_id=925272).
`lidbox` provides this metric as a `tf.keras.Metric` subclass.
Scikit-learn provides other commonly used metrics so there is no need to manually compute those.

In [ ]:
# from lidbox.util import classification_report
# from lidbox.visualize import draw_confusion_matrix


# true_sparse = test_meta.target.to_numpy(np.int32)

# pred_dense = np.stack(test_meta.prediction)
# pred_sparse = pred_dense.argmax(axis=1).astype(np.int32)

# report = classification_report(true_sparse, pred_dense, lang2target)

# for m in ("avg_detection_cost", "avg_equal_error_rate", "accuracy"):
#     print("{}: {:.3f}".format(m, report[m]))
    
# lang_metrics = pd.DataFrame.from_dict({k: v for k, v in report.items() if k in lang2target})
# lang_metrics["mean"] = lang_metrics.mean(axis=1)
# display(lang_metrics.T)

# fig, ax = draw_confusion_matrix(report["confusion_matrix"], lang2target)

## Conclusions

This was an example on deep learning based simple spoken language identification of 4 different languages from the Mozilla Common Voice free speech datasets.
We managed to train a model that adequately recognizes languages spoken by the test set speakers.

However, there is clearly room for improvement.
We did simple random oversampling to balance the language distribution in the training set, but perhaps there are better ways to do this.
We also did not tune optimization hyperparameters or try different neural network architectures or layer combinations.
It might also be possible to increase robustness by audio feature engineering, such as [random FIR filtering](https://www.isca-speech.org/archive/Interspeech_2018/abstracts/1047.html) to simulate microphone differences.